# Working with Tables

When working with Python, there are two ways to do actions on DynamoDB tables - `DynamoDB.Client` and `DynamoDB.Resource`. `DynamoDB.Client` is a low-level client representing DynamoDB and matching SDK APIs as one on one. `DynamoDB.Resource`, on the other hand, provides abstract ways to communicate SDK APIs and more user-friendly.

In this hands on, we're going to use `DynamoDB.Resources` to get our hands dirty. No worries because the two ways provide similar methods and if you get familiar with one of them, I believe you can do the other one easily as well.

Python library, named `boto3`, are all documented [Boto3 Docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html).

## Creating a Table

You will create `Starbucks` table which stores all Starbucks locations in the world by the provided information in [Kaggle](https://www.kaggle.com/starbucks/store-locations). Download `directory.csv` in the given link and uncompress in your local. After that, put the file in the root directory.

When opening the file, you will see the csv-formatted data as follows.

```csv
Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
Starbucks,47323-257470,Korea press center,Joint Venture,"Taepyungro, 25, Taepyungro1ga, Junggu, Seoul",Seoul,11,KR,4520,,GMT+09:00 Asia/Seoul,126.98,37.57
Starbucks,20937-209004,Seoul Womens Univ.,Joint Venture,"621 Hwarang-ro Nowon-gu, Seoul, Korea",Seoul,11,KR,139-774,,GMT+09:00 Asia/Seoul,127.53,37.37
[..]
```

`Store Number` is the unique ID of branches and this can be the good candidate for primary key. We'll use the column as partition key and not set sort key. Here is a simple description for the table.

```
Table Name: Starbucks
Partition Key: StoreNumber
```

As you learnt, there are two capacity modes - provisioned and on-demand. In this hands on, let's use on-demand in all examples to make it simple because our goal is just getting accustomed to the data operation.

In [1]:
# import and get dynamodb resource
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from pprint import pprint
from decimal import Decimal

dynamodb = boto3.resource('dynamodb')

In [31]:
# Create StarbucksLocations table
starbucks = dynamodb.create_table(
    TableName='Starbucks',
    AttributeDefinitions=[
        {
            'AttributeName': 'StoreNumber',
            'AttributeType': 'S'
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'StoreNumber',
            'KeyType': 'HASH'
        }
    ],
    BillingMode='PROVISIONED',
    ProvisionedThroughput={
        'ReadCapacityUnits': 100,
        'WriteCapacityUnits': 100
    }
)

print(starbucks)

dynamodb.Table(name='Starbucks')


In [32]:
# Wait until the table has created
starbucks.wait_until_exists()

## Describing a Table

To view details about a table, use the `DescribeTable` operation. If `DynamoDB.Resource` variable is set, the table variable itself contains the information as attributes.

- archival_summary
- attribute_definitions
- billing_mode_summary
- creation_date_time
- global_secondary_indexes
- global_table_version
- item_count
- key_schema
- latest_stream_arn
- latest_stream_label
- local_secondary_indexes
- provisioned_throughput
- replicas
- restore_summary
- sse_description
- stream_specification
- table_arn
- table_id
- table_name
- table_size_bytes
- table_status

In [33]:
# Refresh the table information
starbucks.load()

In [36]:
# Check various table information
print('Capacity mode:')
pprint(starbucks.billing_mode_summary)
print('Provisioned capacity:')
pprint(starbucks.provisioned_throughput)
print('Key schema:')
pprint(starbucks.key_schema)

Capacity mode:
None
Provisioned capacity:
{'NumberOfDecreasesToday': 0,
 'ReadCapacityUnits': 100,
 'WriteCapacityUnits': 100}
Key schema:
[{'AttributeName': 'StoreNumber', 'KeyType': 'HASH'}]


## Updating a Table

Assume that you want to change the table's capacity mode from on-demand to provisioned mode. The mode is changeable and the change is allowed once in 24 hours.

Here is a sample command to update the table's capacity mode.

In [37]:
# Change capacity mode to provisioned
# It will take roughly 5 mins to complete
starbucks = starbucks.update(
    BillingMode='PAY_PER_REQUEST'
)

In [53]:
# Reload table information and check capacity mode
starbucks.load()

print('Capacity mode:')
pprint(starbucks.billing_mode_summary)
print('Provisioned capacity:')
pprint(starbucks.provisioned_throughput)
print('Table status:')
pprint(starbucks.table_status)

Capacity mode:
{'BillingMode': 'PAY_PER_REQUEST',
 'LastUpdateToPayPerRequestDateTime': datetime.datetime(2020, 10, 2, 16, 17, 4, 371000, tzinfo=tzlocal())}
Provisioned capacity:
{'NumberOfDecreasesToday': 0, 'ReadCapacityUnits': 0, 'WriteCapacityUnits': 0}
Table status:
'ACTIVE'


In [52]:
starbucks.wait_until_exists()

As the application developers are requesting that they need a new access pattern to the table. They want to search the table with `Country` and `State#City` attributes.

To accomodate the request, you need to create a GSI as described below.

```
GSI Name: GSI_01_Locations
Partition Key: Country
Sort Key: StateCity (concatenation of State and City, e.g. AZ#Abu Dhabi)
```

In [54]:
# Create a GSI with updating the table
starbucks = starbucks.update(
    AttributeDefinitions=[
        {
            'AttributeName': 'Country',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'StateCity',
            'AttributeType': 'S'
        }
    ],
    GlobalSecondaryIndexUpdates=[
        {
            'Create': {
                'IndexName': 'GSI_01_Locations',
                'KeySchema': [
                    {
                        'AttributeName': 'Country',
                        'KeyType': 'HASH'
                    },
                    {
                        'AttributeName': 'StateCity',
                        'KeyType': 'RANGE'
                    }
                ],
                'Projection': {
                    'ProjectionType': 'ALL'
                }
            }
        }
    ]
)

In [64]:
# Reload table information and check capacity mode
starbucks.load()

print('Index status:')
pprint(starbucks.global_secondary_indexes)

Index status:
[{'IndexArn': 'arn:aws:dynamodb:ap-northeast-2:886100642687:table/Starbucks/index/GSI_01_Locations',
  'IndexName': 'GSI_01_Locations',
  'IndexSizeBytes': 0,
  'IndexStatus': 'ACTIVE',
  'ItemCount': 0,
  'KeySchema': [{'AttributeName': 'Country', 'KeyType': 'HASH'},
                {'AttributeName': 'StateCity', 'KeyType': 'RANGE'}],
  'Projection': {'ProjectionType': 'ALL'},
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
                            'ReadCapacityUnits': 0,
                            'WriteCapacityUnits': 0}}]


## Deleting a Table

You can remove an unused table with the `DeleteTable` operation. It is unrecoverable.

In [ ]:
# Just check the command, do not delete it actually
starbucks.delete()